In [41]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

In [3]:
dataJan = pd.read_parquet('/content/drive/MyDrive/D:/yellow_tripdata_2023-01.parquet')

Q1 - Downloading the data

In [7]:
dataJan.shape

(3066766, 19)

In [26]:
dataJan

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.30,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.90,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.90,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.10,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.616667
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.40,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066761,2,2023-01-31 23:58:34,2023-02-01 00:12:33,NaN,3.05,NaN,None,107,48,0,15.80,0.00,0.5,3.96,0.0,1.0,23.76,NaN,NaN,13.983333
3066762,2,2023-01-31 23:31:09,2023-01-31 23:50:36,NaN,5.80,NaN,None,112,75,0,22.43,0.00,0.5,2.64,0.0,1.0,29.07,NaN,NaN,19.450000
3066763,2,2023-01-31 23:01:05,2023-01-31 23:25:36,NaN,4.67,NaN,None,114,239,0,17.61,0.00,0.5,5.32,0.0,1.0,26.93,NaN,NaN,24.516667
3066764,2,2023-01-31 23:40:00,2023-01-31 23:53:00,NaN,3.15,NaN,None,230,79,0,18.15,0.00,0.5,4.43,0.0,1.0,26.58,NaN,NaN,13.000000


Q2 - Computing duration

In [9]:
dataJan['duration'] = dataJan['tpep_dropoff_datetime'] - dataJan['tpep_pickup_datetime']

In [13]:
dataJan['duration'] = dataJan['duration'].dt.total_seconds() / 60

In [15]:
dataJan['duration'].std()

42.59435124195458

Q3 - Dropping outliers

In [24]:
dataJan[(dataJan['duration'] > 1) & (dataJan['duration'] < 60)].shape[0] / dataJan.shape[0] * 100

98.11146334607858

In [25]:
dataJan = dataJan[(dataJan['duration'] > 1) & (dataJan['duration'] < 60)]

Q4 - One-hot encoding

In [34]:
from sklearn.feature_extraction import DictVectorizer

In [30]:
categorical = ['PULocationID', 'DOLocationID']
dataJan[categorical] = dataJan[categorical].astype(str)

<ipython-input-30-5e0855246d25>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataJan[categorical] = dataJan[categorical].astype(str)


In [31]:
train_dicts = dataJan[categorical].to_dict(orient='records')

In [32]:
train_dicts

[{'PULocationID': '161', 'DOLocationID': '141'},
 {'PULocationID': '43', 'DOLocationID': '237'},
 {'PULocationID': '48', 'DOLocationID': '238'},
 {'PULocationID': '138', 'DOLocationID': '7'},
 {'PULocationID': '107', 'DOLocationID': '79'},
 {'PULocationID': '161', 'DOLocationID': '137'},
 {'PULocationID': '239', 'DOLocationID': '143'},
 {'PULocationID': '142', 'DOLocationID': '200'},
 {'PULocationID': '164', 'DOLocationID': '236'},
 {'PULocationID': '141', 'DOLocationID': '107'},
 {'PULocationID': '234', 'DOLocationID': '68'},
 {'PULocationID': '79', 'DOLocationID': '264'},
 {'PULocationID': '164', 'DOLocationID': '143'},
 {'PULocationID': '138', 'DOLocationID': '33'},
 {'PULocationID': '33', 'DOLocationID': '61'},
 {'PULocationID': '79', 'DOLocationID': '186'},
 {'PULocationID': '90', 'DOLocationID': '48'},
 {'PULocationID': '113', 'DOLocationID': '255'},
 {'PULocationID': '237', 'DOLocationID': '239'},
 {'PULocationID': '143', 'DOLocationID': '229'},
 {'PULocationID': '137', 'DOLocat

In [35]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [36]:
X_train.shape

(3008849, 515)

Q5 -Training a model

In [39]:
Y_train = dataJan['duration'].values

In [43]:
model = LinearRegression()
model.fit(X_train, Y_train)

y_pred = model.predict(X_train)



In [44]:
print(mean_squared_error(Y_train, y_pred, squared=False))

7.647511913909091


Q6 - Evaluating the model

In [51]:
dataFeb = pd.read_parquet('/content/drive/MyDrive/D:/yellow_tripdata_2023-02.parquet')
dataFeb['duration'] = dataFeb['tpep_dropoff_datetime'] - dataFeb['tpep_pickup_datetime']
dataFeb['duration'] = dataFeb['duration'].dt.total_seconds() / 60
dataFeb = dataFeb[(dataFeb['duration'] > 1) & (dataFeb['duration'] < 60)]


In [52]:
predict_dicts = dataFeb[categorical].to_dict(orient='records')

In [53]:
X_val = dv.transform(predict_dicts)

In [54]:
Y_val = dataFeb['duration'].values

In [59]:
y_pred = model.predict(X_val)

In [60]:
print(mean_squared_error(Y_val, y_pred, squared=False))

13.324206146440376
